In [144]:
# | default_exp routes.user

In [145]:
# %pip install httpx --upgrade
# %pip install domolibrary --upgrade

In [146]:
# | exporti
import os

from enum import Enum
from typing import List

import datetime as dt

import base64
import asyncio
import httpx


import domolibrary.utils.DictDot as dd
from domolibrary.utils.convert import test_valid_email
import domolibrary.client.get_data as gd
import domolibrary.client.ResponseGetData as rgd
import domolibrary.client.DomoAuth as dmda
import domolibrary.client.DomoError as de

import domolibrary.utils.chunk_execution as ce
import domolibrary.utils.Image as uimg

In [147]:
# | hide
import os
import pandas as pd
import base64

# User Route Errors


In [148]:
# | export
class User_CrudError(de.DomoError):
    def __init__(
        self,
        domo_instance,
        function_name,
        status,
        message,
        parent_class=None,
        entity_id=None,
    ):
        super().__init__(
            domo_instance=domo_instance,
            function_name=function_name,
            status=status,
            message=message,
            entity_id=entity_id,
            parent_class=parent_class,
        )


class GetUser_Error(de.DomoError):
    def __init__(
        self, domo_instance, status, response, function_name=None, parent_class=None
    ):
        super().__init__(
            message=response,
            status=status,
            function_name=function_name,
            parent_class=parent_class,
            domo_instance=domo_instance,
        )


class ResetPassword_PasswordUsed(de.DomoError):
    def __init__(
        self,
        status,
        domo_instance,
        entity_id=None,
        message="Password used previously",
        function_name=None,
        parent_class=None,
    ):
        super().__init__(
            message=message,
            status=status,
            function_name=function_name,
            entity_id=entity_id,
            domo_instance=domo_instance,
            parent_class=parent_class,
        )


class SearchUser_NoResults(de.DomoError):
    def __init__(
        self,
        domo_instance,
        search_criteria=None,
        function_name=None,
        parent_class=None,
        status=None,
    ):
        search_str = f"- {search_criteria}" if search_criteria else ""

        super().__init__(
            message=f"query {search_str} returned no users",
            function_name=function_name,
            parent_class=parent_class,
            domo_instance=domo_instance,
            status=status,
        )


class DownloadAvatar_Error(de.DomoError):
    def __init__(
        self,
        status,
        response,
        user_id,
        domo_instance,
        function_name=None,
        parent_class=None,
    ):
        super().__init__(
            status=status,
            message=f"unable to download {user_id} - {response}",
            domo_instance=domo_instance,
            parent_class=parent_class,
            function_name=function_name,
        )

# GET Users


In [149]:
# | export
async def get_all_users(
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class=None,
) -> rgd.ResponseGetData:
    """retrieves all users from Domo"""
    url = f"https://{ auth.domo_instance}.domo.com/api/content/v2/users"

    res = await gd.get_data(
        url=url,
        method="GET",
        auth=auth,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise GetUser_Error(
            domo_instance=auth.domo_instance,
            status=res.status,
            response=res.response,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
        )

    return res

#### sample implementation of get_all_users


In [150]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await get_all_users(auth=token_auth)

len(res.response)

warning this token has not been validated by who_am_i, run get_auth_token first


959

# Search Users

This series of functions searchse the v1_users_api. The body of the search api can be retrieved by searching admin in the Domo User interface.


In [151]:
# | export


async def get_by_id(
    user_id,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    return_raw: bool = False,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop=1,
    parent_class=None,
):
    # does not include role_id
    v2_url = f"https://{auth.domo_instance}.domo.com/api/content/v2/users/{user_id}"

    v3_url = f"https://{auth.domo_instance}.domo.com/api/content/v3/users/{user_id}"

    params = {"includeDetails": True}

    res_v2, res_v3 = await asyncio.gather(
        gd.get_data(
            url=v2_url,
            method="GET",
            auth=auth,
            debug_api=debug_api,
            session=session,
            params=params,
            num_stacks_to_drop=debug_num_stacks_to_drop,
            parent_class=parent_class,
        ),
        gd.get_data(
            url=v3_url,
            method="GET",
            auth=auth,
            debug_api=debug_api,
            session=session,
            params=params,
            num_stacks_to_drop=debug_num_stacks_to_drop,
            parent_class=parent_class,
        ),
    )

    if return_raw:
        return res_v2, res_v3

    if res_v2.status == 200 and res_v2.response == "":
        raise SearchUser_NoResults(
            search_criteria=f"user_id {user_id} not found",
            domo_instance=auth.domo_instance,
            status=res_v2.status,
            function_name=f"{res_v2.traceback_details.function_name}-v2_url",
            parent_class=parent_class,
        )

    if not res_v2.is_success:
        raise GetUser_Error(
            domo_instance=auth.domo_instance,
            status=res_v2.status,
            response=res_v2.response,
            function_name=f"{res_v2.traceback_details.function_name}-v2_url",
            parent_class=parent_class,
        )

    if res_v3.status == "404" and res_v3.response == "Not Found":
        raise SearchUser_NoResults(
            domo_instance=auth.domo_instance,
            status=res_v3.status,
            search_criteria=f"user_id {user_id} not found",
            function_name=f"{res_v3.traceback_details.function_name}-v3_url",
            parent_class=parent_class,
        )
    if (
        not res_v3.status == "404" and not res_v3.response == "Not Found"
    ) and not res_v3.is_success:
        raise GetUser_Error(
            domo_instance=auth.domo_instance,
            status=res_v3.status,
            response=res_v3.response,
            function_name=f"{res_v3.traceback_details.function_name}-v3_url",
            parent_class=parent_class,
        )

    res_v2.response.update({"roleId": res_v3.response.get("roleId")})

    return res_v2

#### sample implementation of get_by_id


In [152]:
# eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

try:
    res = await get_by_id(
        user_id=1236,
        auth=token_auth,
        return_raw=False,
    )
except (SearchUser_NoResults, GetUser_Error) as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
🛑  SearchUser_NoResults 🛑 - function: _run-v2_url || status 200 || query - user_id 1236 not found returned no users at domo-community


In [153]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
user_id = os.environ["DOMO_DOJO_USER_ID"]

await get_by_id(user_id=user_id, auth=token_auth, return_raw=False)

warning this token has not been validated by who_am_i, run get_auth_token first


ResponseGetData(status=200, response={'id': 663516735, 'invitorUserId': 1893952720, 'displayName': 'test 3', 'userName': 'test4@test.com', 'emailAddress': 'test33@test.com', 'accepted': False, 'userType': 'USER', 'modified': 1711216167779, 'created': 1679007751, 'active': True, 'pending': True, 'anonymous': True, 'systemUser': False, 'roleId': 810756122}, is_success=True, parent_class=None)

## Search Users API


In [154]:
# | exporti
def process_v1_search_users(
    v1_user_ls: list[dict],  # list of users from v1_users_search API
) -> list[dict]:  # sanitized list of users.
    """sanitizes the response from v1_users_search API and removes unecessary attributes"""

    clean_users = []

    for obj_user in v1_user_ls:
        dd_user = dd.DictDot(obj_user)

        clean_users.append(
            {
                "id": dd_user.id,
                "displayName": dd_user.displayName,
                "roleId": dd_user.roleId,
                "userName": dd_user.userName,
                "emailAddress": dd_user.emailAddress,
            }
        )

    return clean_users

In [155]:
# | export
async def search_users(
    auth: dmda.DomoAuth,
    body: dict,
    loop_until_end: bool = True,  # retrieve all available rows
    limit=200,  # maximum rows to return per request.  refers to PAGINATION
    maximum=100,  # equivalent to the LIMIT or TOP clause in SQL, the number of rows to return total
    suppress_no_results_error: bool = False,
    debug_api: bool = False,
    return_raw: bool = False,
    debug_loop: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class=None,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/users/search"

    offset_params = {"offset": "offset", "limit": "limit"}

    def body_fn(skip, limit, body):
        return {**body, "limit": limit, "offset": skip}

    def arr_fn(res: rgd.ResponseGetData):
        return res.response.get("users")

    res = await gd.looper(
        auth=auth,
        method="POST",
        url=url,
        maximum=maximum,
        limit=limit,
        offset_params=offset_params,
        offset_params_in_body=True,
        loop_until_end=loop_until_end,
        arr_fn=arr_fn,
        body_fn=body_fn,
        body=body,
        debug_api=debug_api,
        debug_loop=debug_loop,
        debug_num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )

    if return_raw:
        return res

    if not res.is_success:
        raise GetUser_Error(
            domo_instance=auth.domo_instance,
            status=res.status,
            response=res.response,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
        )

    if not suppress_no_results_error and len(res.response) == 0:
        raise SearchUser_NoResults(
            domo_instance=auth.domo_instance,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
        )

    res.response = process_v1_search_users(res.response)

    return res

In [156]:
# | export
async def search_users_by_id(
    user_ids: list[str],  # list of user ids to search
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    return_raw: bool = False,
    suppress_no_results_error: bool = False,
    debug_num_stacks_to_drop=2,
    parent_class=None,
) -> dict:  # dict to pass to search v1_users_search_api
    """search v1_users_search_api"""

    user_cn = ce.chunk_list(user_ids, 1000)

    res_ls = await ce.gather_with_concurrency(
        n=6,
        *[
            search_users(
                auth=auth,
                body={
                    # "showCount": true,
                    # "count": false,
                    "includeDeleted": False,
                    "includeSupport": False,
                    "filters": [
                        {
                            "field": "id",
                            "filterType": "value",
                            "values": user_ls,
                            "operator": "EQ",
                        }
                    ],
                },
                debug_api=debug_api,
                return_raw=return_raw,
                suppress_no_results_error=suppress_no_results_error,
                debug_num_stacks_to_drop=debug_num_stacks_to_drop,
                parent_class=parent_class,
            )
            for user_ls in user_cn
        ]
    )

    res = res_ls[-1]

    res.response = [row for ls in [_.response for _ in res_ls] for row in ls]

    return res

In [157]:
# | export
async def search_users_by_email(
    user_email_ls: list[
        str
    ],  # list of user emails to search.  Note:  search does not appear to be case sensitive
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    return_raw: bool = False,
    suppress_no_results_error: bool = False,
    debug_num_stacks_to_drop=2,
    parent_class=None,
) -> dict:  # dict to pass to search v1_users_search_api
    """search v1_users_search_api"""

    user_cn = ce.chunk_list(user_email_ls, 1000)

    res_ls = await ce.gather_with_concurrency(
        n=10,
        *[
            search_users(
                auth=auth,
                body={
                    # "showCount": true,
                    # "count": false,
                    "includeDeleted": False,
                    "includeSupport": False,
                    "limit": 200,
                    "offset": 0,
                    "sort": {"field": "displayName", "order": "ASC"},
                    "filters": [
                        {
                            "filterType": "text",
                            "field": "emailAddress",
                            "text": " ".join(user_ls),
                        }
                    ],
                },
                debug_api=debug_api,
                return_raw=return_raw,
                suppress_no_results_error=suppress_no_results_error,
                debug_num_stacks_to_drop=debug_num_stacks_to_drop,
                parent_class=parent_class,
            )
            for user_ls in user_cn
        ]
    )

    res = res_ls[-1]

    res.response = [row for ls in [_.response for _ in res_ls] for row in ls]
    return res

#### sample implementation of search_users_by_email


In [158]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await search_users_by_email(
    user_email_ls=["domo.com"],
    auth=token_auth,
    return_raw=False,
    debug_api=False,
)

pd.DataFrame(res.response[0:5])

# res.response

warning this token has not been validated by who_am_i, run get_auth_token first


,id,displayName,roleId,userName,emailAddress
0,1216550715,8:26 - go to sleep,2,test4@domo.com,test4@domo.com
1,128618865,9:02 - no really go to bed,2,test6@domo.com,test6@domo.com
2,1833256765,Aaron Schofield,2097317660,aaron.schofield@domo.com,aaron.schofield@domo.com
3,158618860,Aarzoo Dhillon,2,aarzoo.dhillon@domo.com,aarzoo.dhillon@domo.com
4,696468809,Abish Srinath,2097317660,abish.srinath@domo.com,abish.srinath@domo.com


# Publish and Virtual Users

> Virtual Users are necessary for assigning PDP policies when publish jobs are created.


In [159]:
# | export
async def search_virtual_user_by_subscriber_instance(
    auth: dmda.DomoAuth,  # domo auth object
    subscriber_instance_ls: list[str],  # list of subscriber domo instances
    debug_api: bool = False,  # debug API requests
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
) -> rgd.ResponseGetData:  # list of virtual domo users
    """retrieve virtual users for subscriber instances tied to one publisher"""

    url = f"https://{auth.domo_instance}.domo.com/api/publish/v2/proxy_user/domain/"

    body = {
        "domains": [
            f"{subscriber_instance}.domo.com"
            for subscriber_instance in subscriber_instance_ls
        ]
    }

    return await gd.get_data(
        url=url,
        method="POST",
        auth=auth,
        body=body,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

#### sample of search virtual users by subscriber_instance


In [160]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

res = await search_virtual_user_by_subscriber_instance(
    auth=token_auth, subscriber_instance_ls=["domo-community", "test"], debug_api=False
)


pd.DataFrame(res.response)

warning this token has not been validated by who_am_i, run get_auth_token first


,id,publisherDomain,customerId,subscriberDomain,virtualUserId,created,activeSubscriptions
0,44a56146-f422-4175-9c3b-a194f339f9b6,domo-community.domo.com,mmmm-0012-0200,domo-community.domo.com,fc:f230ba95-bc49-4875-a0db-3c7cd58ed3cc,1680594505000,0
1,8b5c26e0-db3a-40f8-855f-6189ac3cf5c8,domo-community.domo.com,mmmm-0012-0200,test.domo.com,fc:e3ce6205-6717-42b4-ba1c-16b5e40eca98,1680594505000,0


# CRUD Routes and User Attributes


In [161]:
# | export


async def create_user(
    auth: dmda.DomoAuth,
    display_name: str,
    email_address: str,
    role_id: int,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    debug_num_stacks_to_drop: int = 1,
    parent_class: str = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/content/v3/users"

    test_valid_email(email_address)

    body = {
        "displayName": display_name,
        "detail": {"email": email_address},
        "roleId": role_id,
    }

    res = await gd.get_data(
        url=url,
        method="POST",
        body=body,
        auth=auth,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )

    if res.status == 400 and res.response == "Bad Request":
        raise User_CrudError(
            domo_instance=auth.domo_instance,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
            status=res.status,
            message=f"{res.response} - does this user (email) already exist?",
        )

    if not res.is_success:
        raise User_CrudError(
            domo_instance=auth.domo_instance,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
            status=res.status,
            message=res.response,
        )

    res.is_success = True
    return res

#### sample implementation of create_user


In [162]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)
try:
    await create_user(
        auth=token_auth,
        display_name="test_and_delete",
        email_address="test279@test.com",
        role_id=132,
    )
except User_CrudError as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
🛑  User_CrudError 🛑 - function: create_user || status 400 || Bad Request - does this user (email) already exist? at domo-community


In [163]:
# | export
async def set_user_landing_page(
    auth: dmda.DomoAuth,
    user_id: str,
    page_id: str,
    debug_api: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
):
    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/landings/target/DESKTOP/entity/PAGE/id/{page_id}/{user_id}"

    res = await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        # body = body,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
    )

    if not res.is_success:
        raise User_CrudError(
            domo_instance=auth.domo_instance,
            entity_id=user_id,
            status=res.status,
            message=res.response,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
        )

    return res

In [164]:
# | export


async def reset_password(
    auth: dmda.DomoAuth,
    user_id: str,
    new_password: str,
    debug_api: bool = False,
    parent_class=None,
    debug_num_stacks_to_drop=1,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/password"

    body = {"domoUserId": user_id, "password": new_password}

    res = await gd.get_data(
        url=url,
        method="PUT",
        auth=auth,
        body=body,
        debug_api=debug_api,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if (
        res.status == 200
        and res.response.get("description", None)
        == "Password has been used previously."
    ):
        raise ResetPassword_PasswordUsed(
            status=res.status,
            entity_id=user_id,
            domo_instance=auth.domo_instance,
            message=res.response["description"].replace(".", ""),
        )

    return res

#### sample implementation reset_password


In [165]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

try:
    await reset_password(
        auth=token_auth, user_id=1681443709, new_password=os.environ["DOJO_PASSWORD"]
    )
except Exception as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first


In [166]:
# | export
async def request_password_reset(
    domo_instance: str,
    email: str,
    locale="en-us",
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    url = f"https://{domo_instance}.domo.com/api/domoweb/auth/sendReset"

    params = {"email": email, "local": locale}

    return await gd.get_data(
        url=url,
        method="GET",
        params=params,
        auth=None,
        debug_api=debug_api,
        session=session,
    )

### User Properties you can PATCH


In [167]:
# | exports
class UserProperty_Type(Enum):
    display_name = "displayName"
    email_address = "emailAddress"
    phone_number = "phoneNumber"
    title = "title"
    department = "department"
    web_landing_page = "webLandingPage"
    web_mobile_landing_page = "webMobileLandingPage"
    role_id = "roleId"
    employee_id = "employeeId"
    employee_number = "employeeNumber"
    hire_date = "hireDate"
    reports_to = "reportsTo"


class UserProperty:
    property_type: UserProperty_Type
    values: str

    def __init__(self, property_type: UserProperty_Type, values: list):
        self.property_type = property_type
        self.values = self._value_to_list(values)

    @staticmethod
    def _value_to_list(values):
        return values if isinstance(values, list) else [values]

    def to_json(self):
        return {
            "key": self.property_type.value,
            "values": self._value_to_list(self.values),
        }

In [168]:
for member in UserProperty_Type:
    print(member)

UserProperty_Type.display_name
UserProperty_Type.email_address
UserProperty_Type.phone_number
UserProperty_Type.title
UserProperty_Type.department
UserProperty_Type.web_landing_page
UserProperty_Type.web_mobile_landing_page
UserProperty_Type.role_id
UserProperty_Type.employee_id
UserProperty_Type.employee_number
UserProperty_Type.hire_date
UserProperty_Type.reports_to


In [169]:
# | export
def generate_patch_user_property_body(user_property_ls: List[UserProperty]):
    return {
        "attributes": [user_property.to_json() for user_property in user_property_ls]
    }

#### sample implementation of generate_user_property_body


In [170]:
user_property_ls = [
    UserProperty(UserProperty_Type.display_name, "jae myong wilson"),
    UserProperty(UserProperty_Type.email_address, "jae@onyxreporting.com"),
    UserProperty(UserProperty_Type.role_id, 1),
]

generate_patch_user_property_body(user_property_ls)

{'attributes': [{'key': 'displayName', 'values': ['jae myong wilson']},
  {'key': 'emailAddress', 'values': ['jae@onyxreporting.com']},
  {'key': 'roleId', 'values': [1]}]}

In [171]:
# | export
async def update_user(
    user_id: str,
    user_property_ls: List[UserProperty],
    auth: dmda.DomoAuth = None,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
    parent_class: str = None,
    debug_num_stacks_to_drop: int = 1,
):
    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/users/{user_id}"

    body = (
        generate_patch_user_property_body(user_property_ls)
        if isinstance(user_property_ls[0], UserProperty)
        else user_property_ls
    )

    res = await gd.get_data(
        url=url,
        method="PATCH",
        auth=auth,
        body=body,
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )

    if not res.is_success:
        raise User_CrudError(
            domo_instance=auth.domo_instance,
            entity_id=user_id,
            status=res.status,
            message=res.response,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
        )

    return res

#### sample implementation of update_user


In [172]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

user_property_ls = [
    UserProperty(UserProperty_Type.display_name, f"test 3 - updated {dt.date.today()}"),
    UserProperty(UserProperty_Type.email_address, "test3@test.com"),
    UserProperty(UserProperty_Type.role_id, 810756122),
]

res = await update_user(
    user_id=1681443709, user_property_ls=user_property_ls, auth=token_auth
)

res.response

warning this token has not been validated by who_am_i, run get_auth_token first


{'attributes': [{'key': 'id', 'values': [1681443709]},
  {'key': 'displayName', 'values': ['test 3 - updated 2024-03-23']},
  {'key': 'userName', 'values': ['test3@test.com']},
  {'key': 'emailAddress', 'values': ['test3@test.com']},
  {'key': 'modified', 'values': [1711222318453]},
  {'key': 'created', 'values': [1664936244000]},
  {'key': 'roleId', 'values': [810756122]},
  {'key': 'isAnonymous', 'values': [False]},
  {'key': 'isSystemUser', 'values': [False]},
  {'key': 'isPending', 'values': [False]},
  {'key': 'isActive', 'values': [True]},
  {'key': 'invitorUserId', 'values': [1893952720]},
  {'key': 'lastActivity', 'values': [1705960029994], 'source': 'domo'},
  {'key': 'avatarKey', 'values': ['/api/content/v1/avatar/USER/1681443709']}],
 'id': 1681443709,
 'displayName': 'test 3 - updated 2024-03-23',
 'roleId': 810756122,
 'userName': 'test3@test.com',
 'emailAddress': 'test3@test.com'}

### Avatars

In [173]:
# | export
@gd.route_function
async def download_avatar(
    user_id,
    auth: dmda.DomoAuth,
    pixels: int = 300,
    folder_path="./images",
    img_name=None,
    is_download_image: bool = True,
    debug_api: bool = False,
    return_raw: bool = False,
    parent_class: str = None,
    debug_num_stacks_to_drop=1,
    session: httpx.AsyncClient = None,
):
    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/avatar/USER/{user_id}?size={pixels}"

    res = await gd.get_data_stream(
        url=url,
        method="GET",
        auth=auth,
        debug_api=debug_api,
        headers={"accept": "image/png;charset=utf-8"},
        num_stacks_to_drop=debug_num_stacks_to_drop,
        parent_class=parent_class,
        session=session,
    )

    if return_raw:
        return res

    if res.status != 200:
        raise DownloadAvatar_Error(
            status=res.status,
            response=res.response,
            user_id=user_id,
            domo_instance=auth.domo_instance,
            parent_class=parent_class,
            function_name=res.traceback_details.function_name,
        )

    if is_download_image:
        if not os.path.exists(folder_path):
            os.mkdir(folder_path)

        if img_name:
            img_name = img_name.replace(".png", "")

        img_name = f"{img_name or user_id}.png"

        file_path = os.path.join(folder_path, img_name)

        with open(file_path, "wb") as out_file:
            out_file.write(res.response)

    res.is_success = True

    return res

#### sample implementation of downoad_avatar


In [174]:
# | eval : false
token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

user_id = 1833256765

await download_avatar(
    user_id=user_id,
    auth=token_auth,
    debug_api=False,
    folder_path="../test",
    is_download_image=True,
    img_name="route_sample.png",
)

warning this token has not been validated by who_am_i, run get_auth_token first


ResponseGetData(status=200, response=bytearray(b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x01,\x08\x02\x00\x00\x00\xf6\x1f\x19"\x00\x00\x80\x00IDATx^\x9c\xbd\t\x8f\xe4\xd8\x9e\xddW\xdf\xce0dX\x90\x8c\x11l\xcbc\x0b\xb2\xa114\xb0-X#K\x98\xd1\xbc\xd7\xaf\xbb\xa7\xf7\xda+\xf7\x8c}\x8f`\x04\xc9\xe0\xbe3H\x06\xc9\xd8\xb7\xdc\xaa\xaa\xe7\t\xfe&>\xff{#"\xb3\xaa\xfb\xbdyc\xe0\x80`FEeF\x92\xfc\xdds\xfew\xcbg\xe7\x15\xe9\xba\xa1\xd6\xfbnW\x89\x06F\n\t\xfa\xb8\xa3&m\xa6\x8e\x96v\x8d\xbcgN\xa0\xae5\xeb\xd9\x8b\xbe\xbb\xea\xbb\xeb\x9e\xbbn[\x8b\xa61\xabi\xd0\xbca,Z\xd6\xaam\xe3\xc5e\xdb\x98\x0b\xc6\xa4\xaf\x8d\xdbbPk\x9b\xd7\xd5\xe1YAx}\xd6\xfe\xf1M\xed\x9b\x17\xe5\xaf\x7f,@\xdf</}\xff\xaa\xfa\xe2\xa4\xf5\xe6R8+\r\xaf\xebF\xa5\xeb5\xa5\x18?\x0e\xffW0\xa6\x821\xeb\xe9S\xa8o\xce\x07\xd6B\xb4\x97\xa2\x8d\xe3\xfc\xa9$g\xf1T\xb2\xbb\xfc\xc7j\xe8.\x15g\xaf\xa1\xbdx*\xd9\x9a\x7f\xa6?\xf2O\x929\xe3\x12\xadY\xdf\x9a\t\xc6\xa2\xc7\xd45\x96\x1ds\xd5&\xad;\xd6\xb6c\xed\xdaL\x1d\xfb\x86\xabk\xef\

In [175]:
# | export


def generate_avatar_bytestr(img_bytestr, img_type):
    if isinstance(img_bytestr, str):
        img_bytestr = img_bytestr.encode("utf-8")

    if not uimg.isBase64(img_bytestr):
        img_bytestr = base64.b64encode(img_bytestr)

    img_bytestr = img_bytestr.decode("utf-8")

    html_encoding = f"data:image/{img_type};base64,"

    if not img_bytestr.startswith(html_encoding):
        img_bytestr = html_encoding + img_bytestr

    return img_bytestr


@gd.route_function
async def upload_avatar(
    auth: dmda.DomoAuth,
    user_id: int,
    img_bytestr: bytes,
    img_type: str,  #'jpg or png'
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    session: httpx.AsyncClient = None,
    parent_class: str = None,
):
    url = f"https://{auth.domo_instance}.domo.com/api/content/v1/avatar/bulk"

    body = {
        "base64Image": generate_avatar_bytestr(img_bytestr, img_type),
        "encodedImage": generate_avatar_bytestr(img_bytestr, img_type),
        "isOpen": False,
        "entityIds": [user_id],
        "entityType": "USER",
    }

    # return body

    res = await gd.get_data(
        url=url,
        method="POST",
        body=body,
        session=session,
        debug_api=debug_api,
        num_stacks_to_drop=debug_num_stacks_to_drop,
        auth=auth,
        parent_class=None,
    )

    if not res.is_success:
        raise User_CrudError(
            domo_instance=auth.domo_instance,
            entity_id=user_id,
            status=res.status,
            message=res.response,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
        )

    return res

In [176]:
# | eval : false
# import base64

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

user_id = 128618865
img_path = "../classes/images/128618865.png"

with open(img_path, "rb") as f:
    bytestr = f.read()

print(type(bytestr))

await upload_avatar(
    user_id=user_id,
    auth=token_auth,
    debug_api=False,
    img_bytestr=bytestr,
    img_type="jpg",
)

<class 'bytes'>
warning this token has not been validated by who_am_i, run get_auth_token first


ResponseGetData(status=200, response={'channel': 'avatar_bulk_upload-USER_5dda3c05-25b5-4beb-a517-e25b99c27335', 'validatedEvent': 'avatar_bulk_upload-validated_file_USER_5dda3c05-25b5-4beb-a517-e25b99c27335', 'processedEvent': 'avatar_bulk_upload-user_processed_USER_5dda3c05-25b5-4beb-a517-e25b99c27335', 'completeEvent': 'avatar_bulk_upload-complete_USER_5dda3c05-25b5-4beb-a517-e25b99c27335', 'jobId': '140'}, is_success=True, parent_class=None)

# Remove User route

In [177]:
# | export


async def delete_user(
    auth: dmda.DomoAuth,
    user_id: str,
    debug_api: bool = False,
    debug_num_stacks_to_drop=1,
    parent_class: str = None,
    session: httpx.AsyncClient = None,
) -> rgd.ResponseGetData:
    url = f"https://{auth.domo_instance}.domo.com/api/identity/v1/users/{user_id}"

    if debug_api:
        print(url)

    res = await gd.get_data(
        auth=auth,
        url=url,
        method="DELETE",
        debug_api=debug_api,
        session=session,
        parent_class=parent_class,
        num_stacks_to_drop=debug_num_stacks_to_drop,
    )
    if not res.is_success:
        raise User_CrudError(
            domo_instance=auth.domo_instance,
            function_name=res.traceback_details.function_name,
            parent_class=parent_class,
            status=res.status,
            message=res.response,
        )

    return res

#### sample implementation of Delete User

In [178]:
# | eval : false

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

user_id = 554258860

try:
    await delete_user(user_id=user_id, auth=token_auth, debug_api=False)

except User_CrudError as e:
    print(e)

warning this token has not been validated by who_am_i, run get_auth_token first
🛑  User_CrudError 🛑 - function: delete_user || status 404 || Not Found at domo-community


In [179]:
# | hide
import nbdev

nbdev.nbdev_export("./user.ipynb")